In [24]:
#Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
#Carga desde un archivo Excel omitiendo las primeras 4 filas
data = pd.read_excel(r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Valores_Nulos\Actividad 3.2\Gastos y costos 20-23.xlsx", sheet_name= "2022" ,skiprows=3)
data.head(5)

,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,TC,Importe,IVA,RET ISR,RET IVA,Otros,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,1.0,5551.38,888.22,0.0,0.0,NaN,6439.6,6439.6,I,Vigente,PE02-ENE22
1,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,1.0,5551.98,888.32,0.0,0.0,NaN,6440.3,6440.3,I,Vigente,PE03-ENE22
2,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,Servicios de Facturación,PUE,1.0,0.00,0.00,0.0,0.0,NaN,0.0,0.0,I,Vigente,NaN
3,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,NaN,1.0,0.00,0.00,0.0,0.0,NaN,0.0,0.0,P,Vigente,NaN
4,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,NaN,1.0,0.00,0.00,0.0,0.0,NaN,0.0,0.0,P,Vigente,NaN


In [26]:
#Exploración inicial de los datos
print("Dimensiones:", data.shape)
print("\nTipos de datos:")
print(data.dtypes)
print("\nValores nulos por columna:")
print(data.isnull().sum())


Dimensiones: (2577, 19)

Tipos de datos:
Fecha          datetime64[ns]
Folio                  object
UUID                   object
RFC                    object
Proveedor              object
TIPO GASTO             object
Descripción            object
MP                     object
TC                    float64
Importe               float64
IVA                   float64
RET ISR               float64
RET IVA               float64
Otros                 float64
TOTAL MX              float64
TOTAL SAT             float64
Tipo                   object
Status                 object
Poliza                 object
dtype: object

Valores nulos por columna:
Fecha             0
Folio           102
UUID              0
RFC               0
Proveedor         0
TIPO GASTO        0
Descripción       0
MP              553
TC              636
Importe           0
IVA               0
RET ISR           0
RET IVA           0
Otros          2577
TOTAL MX          0
TOTAL SAT         0
Tipo              0
Status 

In [27]:
# Copia del dataset para limpiar
data_limpio = data.copy()

In [28]:
#Remover columna "Otros" (puro nulo y sin aporte de información)
data_limpio.drop(columns=["Otros"], inplace=True)
print("Columna 'Otros' eliminada correctamente de data_limpio ✅")

Columna 'Otros' eliminada correctamente de data_limpio ✅


In [29]:
#Tratamiento de valores nulos en la columna Folio (Remplazar por folio para datos faltantes "999000")
data_limpio["Folio"] = data_limpio["Folio"].fillna(999000)
print("Valores nulos en 'Folio' reemplazados por 999000 ✅")

Valores nulos en 'Folio' reemplazados por 999000 ✅


In [30]:
#Tratamiento de valores nulos en la columna MP
# Paso 1: Reemplazar con 'PGO' cuando la Descripción sea 'Pago'
condicion = (data_limpio["MP"].isna()) & (data_limpio["Descripción"] == "Pago")
data_limpio.loc[condicion, "MP"] = "PGO"
print(f"Reemplazos realizados con 'PGO' cuando Descripción = 'Pago': {condicion.sum()} ✅")

# Paso 2: Reemplazar el/los nulos restantes con la moda de MP
nulos_antes = int(data_limpio["MP"].isna().sum())
if nulos_antes > 0:
    moda_mp = data_limpio["MP"].mode(dropna=True).iloc[0]
    data_limpio["MP"] = data_limpio["MP"].fillna(moda_mp)
    nulos_despues = int(data_limpio["MP"].isna().sum())
    imputados_moda = nulos_antes - nulos_despues
    print(f"Últimos nulos en 'MP' imputados con la moda: {moda_mp}. Registros imputados: {imputados_moda} ✅")
else:
    print("Ya no quedaban nulos en 'MP' después del reemplazo con 'PGO'")

Reemplazos realizados con 'PGO' cuando Descripción = 'Pago': 552 ✅
Últimos nulos en 'MP' imputados con la moda: PUE. Registros imputados: 1 ✅


In [31]:
#Tratamiento de valores nulos en la columna TC
nulos_antes = int(data_limpio["TC"].isna().sum())
data_limpio["TC"] = data_limpio["TC"].fillna(0)
nulos_despues = int(data_limpio["TC"].isna().sum())
print(f"Nulos en 'TC' antes: {nulos_antes}, después: {nulos_despues} ✅")

Nulos en 'TC' antes: 636, después: 0 ✅


In [32]:
#Tratamiento de valores nulos en la columna Poliza
nulos_antes = int(data_limpio["Poliza"].isna().sum())
data_limpio["Poliza"] = data_limpio["Poliza"].fillna("SIN-POLIZA")
nulos_despues = int(data_limpio["Poliza"].isna().sum())
print(f"Nulos en 'Poliza' antes: {nulos_antes}, después: {nulos_despues} ✅")

Nulos en 'Poliza' antes: 801, después: 0 ✅


In [33]:
#Exploración final de los datos
print("Dimensiones:", data_limpio.shape)
print("\nValores nulos por columna:")
print(data_limpio.isnull().sum())

Dimensiones: (2577, 18)

Valores nulos por columna:
Fecha          0
Folio          0
UUID           0
RFC            0
Proveedor      0
TIPO GASTO     0
Descripción    0
MP             0
TC             0
Importe        0
IVA            0
RET ISR        0
RET IVA        0
TOTAL MX       0
TOTAL SAT      0
Tipo           0
Status         0
Poliza         0
dtype: int64


In [34]:
# Revisión final del dataset limpio
data_limpio

,Fecha,Folio,UUID,RFC,Proveedor,TIPO GASTO,Descripción,MP,TC,Importe,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT,Tipo,Status,Poliza
0,2022-01-01,280108,9DA0E63F-1D47-441F-93D7-C2E05AE4D35E,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 18/36 de 03/12/...,PPD,1.0,5551.38,888.22,0.0,0.0,6439.60,6439.60,I,Vigente,PE02-ENE22
1,2022-01-01,280109,7E00E776-A839-4D57-B291-411109164B2F,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 18/36 de 03/12/...,PPD,1.0,5551.98,888.32,0.0,0.0,6440.30,6440.30,I,Vigente,PE03-ENE22
2,2022-01-01,007817297284,BB6585C1-3B54-4130-AFD0-1867C3764821,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,Servicios de Facturación,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,SIN-POLIZA
3,2022-01-01,5595,B5E4B4EB-6C2F-11EC-BB9D-00155D014009,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,PGO,1.0,0.00,0.00,0.0,0.0,0.00,0.00,P,Vigente,SIN-POLIZA
4,2022-01-01,5614,BB6756D1-6CCD-11EC-947D-00155D014407,ZEGC530520V4A,CARLOS ENRIQUE ZERMEÑO GUTIERREZ,COMPRAS,Pago,PGO,1.0,0.00,0.00,0.0,0.0,0.00,0.00,P,Vigente,SIN-POLIZA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,2022-12-01,863376,E90DA089-3C3B-45C8-A19C-B8810D222F60,BIN940223KE0,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",COMISION BANCARIA,COMISIONES E INTERESES,PUE,1.0,0.00,0.00,0.0,0.0,0.00,0.00,I,Vigente,SIN-POLIZA
2573,2022-12-01,1092957089,D720ED7C-F370-4236-B269-8873ABB78745,BMN930209927,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,COMISION BANCARIA,SERVICIOS BANCARIOS,PUE,1.0,17310.30,516.32,0.0,0.0,17826.62,17826.62,I,Vigente,SIN-POLIZA
2574,2022-12-01,4,0AFD3EB5-F3DD-43EB-B9FA-79D793CFF417,AAE6411055P8,Aesa Articulos Electricos S.A de C.V,COMPRAS,Pago,PGO,0.0,0.00,0.00,0.0,0.0,0.00,0.00,P,Vigente,SIN-POLIZA
2575,2022-12-01,16,E3CA1DA8-581A-5844-80D2-CF0317B3BBEA,AAE6411055P8,AESA ARTICULOS ELECTRICOS,COMPRAS,Pago,PGO,0.0,0.00,0.00,0.0,0.0,0.00,0.00,P,Vigente,SIN-POLIZA


In [35]:
#Exportar el DataFrame limpio a CSV
ruta_salida = r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Valores_Nulos\Actividad 3.2\2022_limpio.csv"

data_limpio.to_csv(ruta_salida, index=False, encoding="utf-8-sig")

print(f"Archivo exportado correctamente en: {ruta_salida} ✅")


Archivo exportado correctamente en: C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Valores_Nulos\Actividad 3.2\2022_limpio.csv ✅
